In [ ]:
import sys
import os

sys.path.append('/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent')

from entity_indexing.endpoint_loader import query_sparql

In [ ]:
ground_truth_query = """PREFIX CHEBI: <http://purl.obolibrary.org/obo/CHEBI_>
PREFIX rh: <http://rdf.rhea-db.org/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX up: <http://purl.uniprot.org/core/>


SELECT 
    DISTINCT
        ?uniprot
        ?uniprotID
        ?recname
        ?gene
        ?chebi
        ?uniprotName
WHERE {
  SERVICE <https://sparql.rhea-db.org/sparql> {
     VALUES (?chebi) { (CHEBI:32395) }
     ?rhea rh:side/rh:contains/rh:compound ?compound .
     ?compound rh:chebi ?chebi .
     ?chebi up:name ?uniprotName .
  }
  ?uniprot up:annotation/up:catalyticActivity/up:catalyzedReaction ?rhea .
  ?uniprot up:mnemonic ?uniprotID .
  ?uniprot up:recommendedName/up:fullName ?recname .
  OPTIONAL {?uniprot up:encodedBy/skos:prefLabel ?gene .}
}
LIMIT 100
"""

ground_truth_endpoint = "https://sparql.uniprot.org/sparql/"

In [ ]:
predicted_query = """PREFIX obo: <http://purl.obolibrary.org/obo/>
PREFIX rhea: <http://rdf.rhea-db.org/reaction/>

SELECT DISTINCT ?protein ?reaction ?ecnumber
WHERE {
    # Identify the CHEBI ID for arachidonate
    VALUES ?chebi {obo:CHEBI_32395}
    
    # Query Rhea to find reactions associated with arachidonate
    SERVICE <https://sparql.rhea-db.org/sparql> {
        ?reaction rhea:side ?side .
        ?side rhea:contains ?chebi .
        ?reaction rhea:catalyzedBy ?protein .
    }
    
    # Retrieve EC numbers associated with the reactions
    ?reaction obo:RO_0000057 ?ecnumber .
}"""

predicted_endpoint = "https://sparql.uniprot.org/sparql/"


In [ ]:
bindings_ground_truth = query_sparql(ground_truth_query, ground_truth_endpoint)["results"]["bindings"]
print(bindings_ground_truth)

In [ ]:
results_predicted = query_sparql(predicted_query, target_endpoint)
print(results_predicted)

In [ ]:
import pandas as pd

df = pd.DataFrame(results_ground_truth)

display(df.head())

In [ ]:
extracted_data = []

for row_binding in bindings_ground_truth:
    processed_row = {}
    for var_name, value_dict in row_binding.items():
        if isinstance(value_dict, dict) and 'value' in value_dict:
            processed_row[var_name] = value_dict['value']
        else:
            processed_row[var_name] = None
    extracted_data.append(processed_row)

print(extracted_data)

In [ ]:
df = pd.DataFrame(extracted_data)

display(df.head())

In [ ]:
def result_score(ground_truth_query, ground_truth_endpoint, predicted_query, predicted_endpoint):
    
    ground_truth_data, predicted_data = format_query_result_dataframe(ground_truth_query, ground_truth_endpoint, predicted_query, predicted_endpoint)
    
    metrics = {
        ground_t
    }



In [ ]:
def result_score(ground_truth_query, ground_truth_endpoint, predicted_query, predicted_endpoint):
    
    ground_truth_data, predicted_data = format_query_result_dataframe(ground_truth_query, ground_truth_endpoint, predicted_query, predicted_endpoint)
    
    metrics = {
        ground_t
    }



In [ ]:
ground_truth_data, predicted_data = format_query_result_dataframe(ground_truth_query, ground_truth_endpoint, predicted_query, predicted_endpoint)

In [ ]:
sys.path.append('/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent')

from experiments.utilities.format import process_specific_datasets_and_files

In [ ]:
directory = process_specific_datasets_and_files(endpoint_files_map={"Uniprot": [], "Rhea": [], "SwissLipids": []}, output_dir="endpoint_parse_testing_ground_truth")

In [ ]:


test_set = [
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/118_biosodafrontend_rat_brain_human_cancer",
    "natural_language_question": "What are the Homo sapiens genes associated with cancer and their orthologs expressed in the Rattus norvegicus brain?",
    "query": "PREFIX up:<http://purl.uniprot.org/core/>\nPREFIX taxon:<http://purl.uniprot.org/taxonomy/>\nPREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>\nPREFIX orth:<http://purl.org/net/orth#>\nPREFIX dcterms:<http://purl.org/dc/terms/>\nPREFIX obo:<http://purl.obolibrary.org/obo/>\nPREFIX lscr:<http://purl.org/lscr#>\nPREFIX genex:<http://purl.org/genex#>\nPREFIX sio: <http://semanticscience.org/resource/>\nSELECT ?gene ?orthologous_protein2 WHERE {\n  {\n    SELECT ?protein1 WHERE {\n      ?protein1 a up:Protein;\n        up:organism/up:scientificName 'Homo sapiens' ;\n        up:annotation ?annotation .\n      ?annotation rdfs:comment ?annotation_text.\n      ?annotation a up:Disease_Annotation .\n      FILTER CONTAINS (?annotation_text, \"cancer\")\n    }\n  }\n  SERVICE <https://sparql.omabrowser.org/sparql/> {\n    SELECT ?orthologous_protein2 ?protein1 ?gene WHERE {\n      ?protein_OMA a orth:Protein .\n      ?orthologous_protein2 a orth:Protein .\n      ?cluster a orth:OrthologsCluster .\n      ?cluster orth:hasHomologousMember ?node1 .\n      ?cluster orth:hasHomologousMember ?node2 .\n      ?node2 orth:hasHomologousMember* ?protein_OMA .\n      ?node1 orth:hasHomologousMember* ?orthologous_protein2 \n      .?orthologous_protein2 orth:organism/obo:RO_0002162/up:scientificName 'Rattus norvegicus' .\n      ?orthologous_protein2 sio:SIO_010079 ?gene .\n      ?protein_OMA lscr:xrefUniprot ?protein1 .\n      FILTER(?node1 != ?node2)\n    }\n  }\n  SERVICE <https://www.bgee.org/sparql/> {\n    ?gene genex:isExpressedIn ?anatEntity .\n    ?anatEntity rdfs:label 'brain' .\n    ?gene orth:organism ?org . \n    ?org obo:RO_0002162 taxon:10116 .\n  }\n}",
    "target_endpoint": "https://sparql.uniprot.org/sparql",
    "federates_with": "https://sparql.omabrowser.org/sparql/",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/118_biosodafrontend_rat_brain_human_cancer.ttl",
    "filename": "118_biosodafrontend_rat_brain_human_cancer.ttl"
  },
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/50",
    "natural_language_question": "Where are the human genes encoding enzymes metabolizing N-acyl\nsphingosines expressed in the human body (federated query, with Rhea and Bgee)",
    "query": "PREFIX CHEBI: <http://purl.obolibrary.org/obo/CHEBI_>\nPREFIX genex: <http://purl.org/genex#>\nPREFIX lscr: <http://purl.org/lscr#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX taxon: <http://purl.uniprot.org/taxonomy/>\nPREFIX up: <http://purl.uniprot.org/core/>\n\nSELECT\n  DISTINCT\n    ?chebi\n    ?reaction\n    ?protein ?ensemblGene\n    ?anatomicEntityLabel\n    ?anatomicEntity\nWHERE {\n  SERVICE <https://sparql.rhea-db.org/sparql> {\n    ?reaction rdfs:subClassOf rh:Reaction .\n    ?reaction rh:equation ?reactionEquation .\n    ?reaction rh:side ?reactionSide .\n    ?reactionSide rh:contains ?participant .\n    ?participant rh:compound ?compound .\n    ?compound rh:chebi ?chebi .\n    ?chebi rdfs:subClassOf* CHEBI:52639\n}\n  ?protein up:organism taxon:9606 .\n  ?protein up:annotation ?a .\n  ?a a up:Catalytic_Activity_Annotation .\n  ?a up:catalyticActivity ?ca .\n  ?ca up:catalyzedReaction ?reaction .\n  ?protein rdfs:seeAlso / up:transcribedFrom ?ensemblGene .\n\n  SERVICE  <https://www.bgee.org/sparql/> {\n    ?gene genex:isExpressedIn ?anatomicEntity .\n    ?gene lscr:xrefEnsemblGene ?ensemblGene .\n    ?anatomicEntity rdfs:label ?anatomicEntityLabel .\n  }\n}",
    "target_endpoint": "https://sparql.uniprot.org/sparql/",
    "federates_with": "https://sparql.rhea-db.org/sparql",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/50_tissues_where_genes_sphingosines_are_expressed.ttl",
    "filename": "50_tissues_where_genes_sphingosines_are_expressed.ttl"
  },
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/51",
    "natural_language_question": "Find all proteins linked to arachidonate (CHEBI:32395)",
    "query": "PREFIX CHEBI: <http://purl.obolibrary.org/obo/CHEBI_>\nPREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX skos: <http://www.w3.org/2004/02/skos/core#>\nPREFIX up: <http://purl.uniprot.org/core/>\n\n\nSELECT \n    DISTINCT\n        ?uniprot\n        ?uniprotID\n        ?recname\n        ?gene\n        ?chebi\n        ?uniprotName\nWHERE {\n  SERVICE <https://sparql.rhea-db.org/sparql> {\n     VALUES (?chebi) { (CHEBI:32395) }\n     ?rhea rh:side/rh:contains/rh:compound ?compound .\n     ?compound rh:chebi ?chebi .\n     ?chebi up:name ?uniprotName .\n  }\n  ?uniprot up:annotation/up:catalyticActivity/up:catalyzedReaction ?rhea .\n  ?uniprot up:mnemonic ?uniprotID .\n  ?uniprot up:recommendedName/up:fullName ?recname .\n  OPTIONAL {?uniprot up:encodedBy/skos:prefLabel ?gene .}\n}",
    "target_endpoint": "https://sparql.uniprot.org/sparql/",
    "federates_with": "https://sparql.rhea-db.org/sparql",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/51_all_proteins_linked_to_arachidonate.ttl",
    "filename": "51_all_proteins_linked_to_arachidonate.ttl"
  },
  
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/40",
    "natural_language_question": "Retrieve human enzymes that metabolize sphingolipids and are annotated in ChEMBL",
    "query": "PREFIX CHEBI: <http://purl.obolibrary.org/obo/CHEBI_>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX taxon: <http://purl.uniprot.org/taxonomy/>\nPREFIX up: <http://purl.uniprot.org/core/>\n\nSELECT DISTINCT ?protein ?chemblEntry\nWHERE {\n  SERVICE <https://sparql.rhea-db.org/sparql> {\n    ?rhea rdfs:subClassOf rh:Reaction ;\n      rh:side/rh:contains/rh:compound/rh:chebi/rdfs:subClassOf+ CHEBI:26739 .\n  }\n  ?ca up:catalyzedReaction ?rhea .\n  ?protein up:annotation/up:catalyticActivity ?ca ;\n    up:organism taxon:9606 ;\n    rdfs:seeAlso ?chemblEntry .\n  ?chemblEntry up:database <http://purl.uniprot.org/database/ChEMBL> .\n}",
    "target_endpoint": "https://sparql.uniprot.org/sparql/",
    "federates_with": "https://sparql.rhea-db.org/sparql",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/40_human_enzymes_that_metabolize_sphingolipids.ttl",
    "filename": "40_human_enzymes_that_metabolize_sphingolipids.ttl"
  },
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/92_uniprot_bioregistry_iri_translation",
    "natural_language_question": "Translate the global unique identifier for a UniProtKB record into other options using the bioregistry translating endpoint.",
    "query": "PREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX uniprotkb: <http://purl.uniprot.org/uniprot/>\nPREFIX up: <http://purl.uniprot.org/core/>\n\nSELECT \n    ?protein \n    ?otherIdentifier\nWHERE {\n    BIND(uniprotkb:P00750 AS ?protein)\n    ?protein a up:Protein .\n    SERVICE <https://bioregistry.io/sparql> {\n        ?protein owl:sameAs ?otherIdentifier .\n    }\n}",
    "target_endpoint": "https://sparql.uniprot.org/sparql/",
    "federates_with": "https://bioregistry.io/sparql",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/92_uniprot_bioregistry_iri_translation.ttl",
    "filename": "92_uniprot_bioregistry_iri_translation.ttl"
  },
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/49",
    "natural_language_question": "Retrieve the UniProtKB proteins, their catalyzed Rhea reactions, their encoding genes (Ensembl) and the anatomic entities where the genes are expressed (UBERON anatomic entites from Bgee expression data resource).",
    "query": "PREFIX CHEBI: <http://purl.obolibrary.org/obo/CHEBI_>\nPREFIX genex: <http://purl.org/genex#>\nPREFIX lscr: <http://purl.org/lscr#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX taxon: <http://purl.uniprot.org/taxonomy/>\nPREFIX up: <http://purl.uniprot.org/core/>\n\nSELECT DISTINCT ?protein ?ensemblGene ?reaction ?anatomicEntityLabel ?anatomicEntity\nWHERE {\n  # federated query to Rhea enadpoint\n  {\n    SELECT DISTINCT ?reaction WHERE {\n      SERVICE <https://sparql.rhea-db.org/sparql> {\n        ?reaction rdfs:subClassOf rh:Reaction ;\n          rh:equation ?reactionEquation ;\n          rh:side ?reactionSide .\n        ?reactionSide rh:contains ?participant .\n        ?participant rh:compound ?compound .\n        # compound constraint (CHEBI:16113 == cholesterol)\n        ?compound rh:chebi CHEBI:16113 .\n      }\n    }\n  }\n  # taxonomy constraint (taxon:9606 == Homo sapiens)\n  ?protein up:organism taxon:9606 ;\n    up:annotation ?a ;\n    rdfs:seeAlso / up:transcribedFrom ?ensemblGene .\n  ?a a up:Catalytic_Activity_Annotation ;\n    up:catalyticActivity ?ca .\n  ?ca up:catalyzedReaction ?reaction .\n  # federated query to Bgee (expression data)\n  BIND(IRI(REPLACE(STR(?ensemblGene), \"\\\\.[0-9]+$\", \"\")) AS ?ensemblGeneNoVersion)\n  SERVICE <https://www.bgee.org/sparql/> {\n    ?gene lscr:xrefEnsemblGene ?ensemblGeneNoVersion ;\n      genex:isExpressedIn ?anatomicEntity .\n    ?anatomicEntity rdfs:label ?anatomicEntityLabel .\n  }\n}",
    "target_endpoint": "https://sparql.uniprot.org/sparql/",
    "federates_with": "https://sparql.rhea-db.org/sparql",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/49_tissues_where_genes_metabolizing_cholestrol_are_expressed.ttl",
    "filename": "49_tissues_where_genes_metabolizing_cholestrol_are_expressed.ttl"
  },
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/53",
    "natural_language_question": "Find mouse homologs in OMABrowser of human enzymes that catalyze reactions involving sterols (CHEBI:15889). Federating with Rhea and OMABrowser.",
    "query": "PREFIX CHEBI: <http://purl.obolibrary.org/obo/CHEBI_>\nPREFIX lscr: <http://purl.org/lscr#>\nPREFIX orth: <http://purl.org/net/orth#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX taxon: <http://purl.uniprot.org/taxonomy/>\nPREFIX up: <http://purl.uniprot.org/core/>\n\nSELECT\n  DISTINCT\n    ?chebi\n    ?reaction\n    ?humanProtein\n    ?mouseProtein\n    ?cluster \nWHERE {\n  SERVICE <https://sparql.rhea-db.org/sparql> {\n    ?reaction rdfs:subClassOf rh:Reaction .\n    ?reaction rh:side/rh:contains/rh:compound ?compound .\n    ?compound rh:chebi ?chebi .\n    ?chebi rdfs:subClassOf* CHEBI:15889\n  }\n\n  ?humanProtein up:organism taxon:9606 .\n  ?humanProtein up:annotation ?a .\n  ?a a up:Catalytic_Activity_Annotation .\n  ?a up:catalyticActivity ?ca .\n  ?ca up:catalyzedReaction ?reaction .\n\n  SERVICE  <https://sparql.omabrowser.org/sparql> {\n    ?cluster a orth:ParalogsCluster .\n    ?cluster orth:hasHomologousMember ?node1 , ?node2 .\n    ?node1 orth:hasHomologousMember* ?orthoProtein1 .\n    ?node2 orth:hasHomologousMember* ?orthoProtein2 .\n    ?orthoProtein1 lscr:xrefUniprot ?mouseProtein .\n    ?orthoProtein2 lscr:xrefUniprot ?humanProtein .\n    # inTaxon mouse\n    ?orthoProtein1 orth:organism/<http://purl.obolibrary.org/obo/RO_0002162> taxon:10090 . \n  }\n}",
    "target_endpoint": "https://sparql.uniprot.org/sparql/",
    "federates_with": "https://sparql.rhea-db.org/sparql",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/53_mouse_homologs_of_sterol_enzymes_via_omabrowser.ttl",
    "filename": "53_mouse_homologs_of_sterol_enzymes_via_omabrowser.ttl"
  },
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/52",
    "natural_language_question": "Retrieve drugs that target human enzymes involved in sterol metabolism (federated query with Rhea and ChEMBL via IDSM/Elixir czech republic).",
    "query": "PREFIX CHEBI: <http://purl.obolibrary.org/obo/CHEBI_>\nPREFIX cco: <http://rdf.ebi.ac.uk/terms/chembl#>\nPREFIX chebihash: <http://purl.obolibrary.org/obo/chebi#>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX skos: <http://www.w3.org/2004/02/skos/core#>\nPREFIX taxon: <http://purl.uniprot.org/taxonomy/>\nPREFIX up: <http://purl.uniprot.org/core/>\n\nSELECT\n        DISTINCT\n            ?protein\n            ?proteinFullName\n            ?activityType\n            ?standardActivityValue\n            ?standardActivityUnit\n            ?chemblMolecule\n            ?chemlbMoleculePrefLabel\nWHERE\n  {\n  # ChEBI: retrieve members of the ChEBI class ChEBI:15889 (sterol)\n  # Rhea: retrieve the reactions involving these ChEBI as participants\n  SERVICE <https://sparql.rhea-db.org/sparql> {\n    ?reaction rdfs:subClassOf  rh:Reaction ;\n\t      rh:status        rh:Approved ;\n\t      rh:side          ?reactionSide .\n    ?reactionSide\n\t      rh:contains      ?participant .\n    ?participant rh:compound  ?compound\n    {\n      ?compound  rh:chebi  ?chebi .\n      ?chebi (rdfs:subClassOf)+ CHEBI:15889\n    } UNION {\n      ?compound  rh:chebi           ?chebi .\n      ?chebi2   rdfs:subClassOf     ?chebiRestriction .\n      ?chebiRestriction\n\t\ta           owl:Restriction ;\n\t\towl:onProperty      chebihash:has_major_microspecies_at_pH_7_3 ;\n\t\towl:someValuesFrom  ?chebi .\n      ?chebi2 (rdfs:subClassOf)+ CHEBI:15889\n    }\n  }\n  # UniProt: retrieve the human (taxid:9606) enzymes catalyzing these Rhea reactions\n  ?ca       up:catalyzedReaction  ?reaction .\n  ?a        up:catalyticActivity  ?ca .\n  ?protein  up:annotation         ?a ;\n\t    up:organism           taxon:9606 ;\n\t    up:recommendedName    ?proteinRecName .\n  ?proteinRecName\n\t    up:fullName           ?proteinFullName .\n  # Find drugs in wikidata that interact with the UniProt Proteins\n  # ChEMBL: retrieve the corresponding targets and with drugs in clinical phase 4\n  # Via https://idsm.elixir-czech.cz/sparql/\n  SERVICE <https://idsm.elixir-czech.cz/sparql/endpoint/idsm> {\n    ?activity a cco:Activity ;\n      cco:hasMolecule ?chemblMolecule ;\n      cco:hasAssay ?assay ;\n      cco:standardType ?activityType ;\n      cco:standardValue ?standardActivityValue ;\n      cco:standardUnits ?standardActivityUnit .\n    ?chemblMolecule cco:highestDevelopmentPhase ?highestDevelopmentPhase ;\n      rdfs:label ?chemblMoleculeLabel ;\n      skos:prefLabel ?chemlbMoleculePrefLabel .\n    FILTER (?highestDevelopmentPhase > 3)\n    ?assay cco:hasTarget ?target .\n    ?target cco:hasTargetComponent/cco:targetCmptXref ?protein .\n    ?protein a cco:UniprotRef .\n  }\n}",
    "target_endpoint": "https://sparql.uniprot.org/sparql/",
    "federates_with": "https://sparql.rhea-db.org/sparql",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/52_drugs_targeting_sterol_metabolism.ttl",
    "filename": "52_drugs_targeting_sterol_metabolism.ttl"
  },
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/109_uniprot_transporter_in_liver",
    "natural_language_question": "Find human transporter proteins in reviewed UniProtKB/Swiss-Prot, that are expressed in the liver (Uses Bgee and UBERON).",
    "query": "PREFIX genex: <http://purl.org/genex#>\nPREFIX lscr: <http://purl.org/lscr#>\nPREFIX obo: <http://purl.obolibrary.org/obo/>\nPREFIX orth: <http://purl.org/net/orth#>\nPREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX taxon: <http://purl.uniprot.org/taxonomy/>\nPREFIX uberon: <http://purl.obolibrary.org/obo/uo#>\nPREFIX up: <http://purl.uniprot.org/core/>\n\nSELECT\n  ?rhea\n  ?protein\n  ?anat\nWHERE\n{\n  GRAPH <https://sparql.rhea-db.org/rhea> {\n    ?rhea rh:isTransport true .\n  }\n  ?protein up:annotation ?ann .\n  ?protein up:organism taxon:9606 .\n  ?ann up:catalyticActivity ?ca .\n  ?ca up:catalyzedReaction ?rhea .\n  BIND(uberon:0002107 AS ?anat)\n  SERVICE <https://www.bgee.org/sparql/> {\n    ?seq genex:isExpressedIn ?anat .\n    ?seq lscr:xrefUniprot ?protein .\n    ?seq orth:organism ?organism .\n    ?organism obo:RO_0002162 taxon:9606 .\n  }\n}",
    "target_endpoint": "https://sparql.uniprot.org/sparql/",
    "federates_with": "https://www.bgee.org/sparql/",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/109_uniprot_transporter_in_liver.ttl",
    "filename": "109_uniprot_transporter_in_liver.ttl"
  },
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/54",
    "natural_language_question": "Proteins with binding sites for ligands similar to heme",
    "query": "PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX sachem: <http://bioinfo.uochb.cas.cz/rdf/v1.0/sachem#>\nPREFIX up: <http://purl.uniprot.org/core/>\nPREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\n\nSELECT \n  ?protein\n  ?mnemonic\n  ?proteinName\n  ?ligandSimilarityScore\n  ?ligand\nWHERE {\n  SERVICE <https://idsm.elixir-czech.cz/sparql/endpoint/chebi> {\n    ?ssc sachem:compound ?ligand; \n      sachem:score ?ligandSimilarityScore ;\n      sachem:similaritySearch ?sss .\n        # Smiles of Heme\n    ?sss    sachem:query \"CC1=C(CCC([O-])=O)C2=[N+]3C1=Cc1c(C)c(C=C)c4C=C5C(C)=C(C=C)C6=[N+]5[Fe-]3(n14)n1c(=C6)c(C)c(CCC([O-])=O)c1=C2\";\n      sachem:cutoff \"8e-1\"^^xsd:double ;\n      sachem:aromaticityMode sachem:aromaticityDetect ;\n      sachem:similarityRadius 1 ;\n      sachem:tautomerMode sachem:ignoreTautomers .\n  }\n  ?protein up:mnemonic ?mnemonic ;\n    up:recommendedName/up:fullName ?proteinName ;\n    up:annotation ?annotation .\n  ?annotation a up:Binding_Site_Annotation ;\n      up:ligand/rdfs:subClassOf ?ligand .\n}\nORDER BY DESC(?ligandSimilarityScore)",
    "target_endpoint": "https://sparql.uniprot.org/sparql/",
    "federates_with": "https://idsm.elixir-czech.cz/sparql/endpoint/chebi",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/54_binding_sites_for_ligands_similar_to_heme_sachem.ttl",
    "filename": "54_binding_sites_for_ligands_similar_to_heme_sachem.ttl"
  },
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/43",
    "natural_language_question": "Connect patents cited in UniProtKB with those in the patent database at EPO via publication number, whose grant date is more than twenty years in the past.",
    "query": "PREFIX patent: <http://data.epo.org/linked-data/def/patent/>\nPREFIX skos: <http://www.w3.org/2004/02/skos/core#>\nPREFIX up: <http://purl.uniprot.org/core/>\n\nSELECT ?grantDate ?patent ?application ?applicationNo\nWHERE\n{\n    ?citation a up:Patent_Citation ;\n  skos:exactMatch ?patent .\n  BIND(SUBSTR(STR(?patent), 35) AS ?applicationNo)\n  BIND(SUBSTR(STR(?patent), 33, 2) AS ?countryCode)\n  SERVICE <https://data.epo.org/linked-data/query>{\n    ?publication patent:publicationNumber ?applicationNo ;\n      patent:application ?application .\n    ?application patent:grantDate ?grantDate .\n  }\n  BIND((year(now()) - 20) AS ?thisYearMinusTwenty)\n  BIND(year(?grantDate) AS ?grantYear)\n  FILTER(?grantYear < ?thisYearMinusTwenty)\n} ORDER BY ?grantYear",
    "target_endpoint": "https://sparql.uniprot.org/sparql/",
    "federates_with": "https://data.epo.org/linked-data/query",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/43_patents_used_in_uniprot_granted_twenty_years_in_the_past.ttl",
    "filename": "43_patents_used_in_uniprot_granted_twenty_years_in_the_past.ttl"
  },
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/45",
    "natural_language_question": "Retrieve drugs that target human enzymes involved in sterol metabolism (federated query with WikiData and Rhea).",
    "query": "PREFIX CHEBI: <http://purl.obolibrary.org/obo/CHEBI_>\nPREFIX chebihash: <http://purl.obolibrary.org/obo/chebi#>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX taxon: <http://purl.uniprot.org/taxonomy/>\nPREFIX uniprotkb: <http://purl.uniprot.org/uniprot/>\nPREFIX up: <http://purl.uniprot.org/core/>\nPREFIX wdt: <http://www.wikidata.org/prop/direct/>\n\nSELECT DISTINCT ?protein ?proteinFullName ?wikiChemical ?wikiChemicalLabel ?medicalConditionTreatedLabel\nWHERE {\n  # ChEBI: retrieve members of the ChEBI class ChEBI:15889 (sterol)\n  # Rhea: retrieve the reactions involving these ChEBI as participants\n  SERVICE <https://sparql.rhea-db.org/sparql> {\n    ?reaction rdfs:subClassOf rh:Reaction ;\n      rh:status rh:Approved ;\n      rh:side ?reactionSide .\n    ?reactionSide\n      rh:contains ?participant .\n    ?participant rh:compound ?compound\n    {\n      ?compound rh:chebi ?chebi .\n      ?chebi (rdfs:subClassOf)+ CHEBI:15889\n    } UNION {\n      ?compound rh:chebi ?chebi .\n      ?chebi2 rdfs:subClassOf ?chebiRestriction .\n      ?chebiRestriction\n\t\ta owl:Restriction ;\n\t\towl:onProperty chebihash:has_major_microspecies_at_pH_7_3 ;\n\t\towl:someValuesFrom ?chebi .\n      ?chebi2 (rdfs:subClassOf)+ CHEBI:15889\n    }\n  }\n  # UniProt: retrieve the human (taxid:9606) enzymes catalyzing these Rhea reactions\n  ?ca up:catalyzedReaction  ?reaction .\n  ?a up:catalyticActivity  ?ca .\n  ?proteinIRI  up:annotation ?a ;\n    up:organism taxon:9606 ;\n    up:recommendedName ?proteinRecName .\n  ?proteinRecName up:fullName ?proteinFullName .\n  # Find drugs in wikidata that interact with the UniProt Proteins\n  BIND (SUBSTR(STR(?proteinIRI), STRLEN(STR(uniprotkb:))+1) AS ?protein)\n  SERVICE <https://query.wikidata.org/sparql> {\n    ?wp wdt:P352  ?protein .\n    ?wikiChemical wdt:P129 ?wp . # Physically interacts with\n    ?wikiChemical rdfs:label ?wikiChemicalLabel .\n    ?wikiChemical wdt:P2175 ?wmc . # Medical conndition treated\n    ?wmc rdfs:label ?medicalConditionTreatedLabel .\n    FILTER(lang(?medicalConditionTreatedLabel) = 'en')\n    FILTER(lang(?wikiChemicalLabel) = 'en')\n  }\n}",
    "target_endpoint": "https://sparql.uniprot.org/sparql/",
    "federates_with": "https://query.wikidata.org/sparql",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/45_drugs_targeting_human_sterol_metabolism_enzymes.ttl",
    "filename": "45_drugs_targeting_human_sterol_metabolism_enzymes.ttl"
  },
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/42",
    "natural_language_question": "Connect patents cited in UniProtKB with those in the patent database at EPO via publication number.",
    "query": "PREFIX patent: <http://data.epo.org/linked-data/def/patent/>\nPREFIX skos: <http://www.w3.org/2004/02/skos/core#>\nPREFIX up: <http://purl.uniprot.org/core/>\n\nSELECT ?citation ?patent ?application ?applicationNo\nWHERE\n{\n  ?citation a up:Patent_Citation ;\n    skos:exactMatch ?patent .\n  FILTER(CONTAINS(STR(?patent), 'EP'))\n  BIND(SUBSTR(STR(?patent), 35) AS ?applicationNo)\n  SERVICE <https://data.epo.org/linked-data/query>{\n    ?application patent:publicationNumber ?applicationNo\n  }\n}",
    "target_endpoint": "https://sparql.uniprot.org/sparql/",
    "federates_with": "https://data.epo.org/linked-data/query",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/42_connect_patents_to_epo.ttl",
    "filename": "42_connect_patents_to_epo.ttl"
  },
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/67_draft_human_metabolome",
    "natural_language_question": "Generate a draft human metabolome",
    "query": "PREFIX chebislash: <http://purl.obolibrary.org/obo/chebi/>\nPREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX taxon: <http://purl.uniprot.org/taxonomy/>\nPREFIX up: <http://purl.uniprot.org/core/>\n\nSELECT\n?uniprot ?mnemonic ?rhea ?chebi ?smiles ?inchiKey\nWHERE\n{\n  ?uniprot up:annotation/up:catalyticActivity/up:catalyzedReaction ?rhea ;\n        up:organism taxon:9606 ;\n        up:mnemonic ?mnemonic .\n  SERVICE <https://sparql.rhea-db.org/sparql> {\n\t?rhea rh:side/rh:contains/rh:compound ?compound .\n    ?compound (rh:chebi|(rh:reactivePart/rh:chebi)|(rh:underlyingChebi/rh:chebi)) ?chebi .\n   }\n   ?chebi chebislash:smiles ?smiles ;\n          chebislash:inchikey ?inchiKey .\n}",
    "target_endpoint": "https://sparql.uniprot.org/sparql/",
    "federates_with": "https://sparql.rhea-db.org/sparql",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/67_draft_human_metabolome.ttl",
    "filename": "67_draft_human_metabolome.ttl"
  },
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/36",
    "natural_language_question": "Find the orthologous proteins for UniProtKB entry P05067 using the <a href=\"http://www.orthod.org\">OrthoDB database</a>",
    "query": "PREFIX orthodb: <http://purl.orthodb.org/>\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX skos: <http://www.w3.org/2004/02/skos/core#>\nPREFIX uniprotkb: <http://purl.uniprot.org/uniprot/>\nPREFIX up: <http://purl.uniprot.org/core/>\n\nSELECT\n  ?protein\n  ?orthoGroup\n  ?scientificName\n  ?functionComment\n  ?prefferedGeneName\n  ((STRLEN(?value) - ?medianLength) as ?deviationFromMedianLength)\nWHERE\n{\n  uniprotkb:P05067 a up:Protein ;\n        up:organism/up:scientificName ?scientificName ;\n        rdfs:seeAlso ?orthoGroup ;\n        up:encodedBy/skos:prefLabel ?prefferedGeneName ;\n          up:sequence/rdf:value ?value .\n  OPTIONAL {\n    ?protein up:annotation ?functionAnnotation .\n    ?functionAnnotation a up:Function_Annotation ;\n      rdfs:comment ?functionComment .\n  }\n  SERVICE <https://sparql.orthodb.org/sparql>{\n    ?orthoGroup orthodb:ogMedianProteinLength ?medianLength .\n    ?orthoGroup orthodb:hasMember ?xref .\n    ?xref orthodb:xref/orthodb:xrefResource uniprotkb:P05067 .\n  }\n}",
    "target_endpoint": "https://sparql.uniprot.org/sparql/",
    "federates_with": "https://sparql.orthodb.org/sparql",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/36_orthologous_proteins_via_orthodb.ttl",
    "filename": "36_orthologous_proteins_via_orthodb.ttl"
  },
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/60",
    "natural_language_question": "Find human proteins that catalyze reactions where substrates or product have a <a href=\"https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI:35516\">Cholestane skeleton</a>",
    "query": "PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX sachem: <http://bioinfo.uochb.cas.cz/rdf/v1.0/sachem#>\nPREFIX taxon: <http://purl.uniprot.org/taxonomy/>\nPREFIX up: <http://purl.uniprot.org/core/>\n\nSELECT\n  DISTINCT\n    ?protein\n    ?disease\n    ?rhea\n    ?chebi\n    ?omim\nWHERE {\n    # Find complete ChEBIs with a Cholestane skeleton, via the Czech Elixir node IDSM Sachem chemical substructure search.\n    SERVICE <https://idsm.elixir-czech.cz/sparql/endpoint/chebi> {\n      ?chebi sachem:substructureSearch [\n        sachem:query\n\"[C@]12(CCC3CCCC[C@]3(C)[C@@]1([H])CC[C@]1(C)[C@@]([H])([C@@](C)([H])CCCC(C)C)CC[C@@]21[H])[H]\"\n].\n   }\n   # Use the fact that UniProt catalytic activities are annotated using Rhea \n   # Mapping the found ChEBIs to Rhea reactions\n   SERVICE <https://sparql.rhea-db.org/sparql>{\n     ?rhea rh:side/rh:contains/rh:compound/rdfs:subClassOf ?chebi .\n   }\n   # Match the found Rhea reactions with human UniProtKB proteins\n   ?protein up:annotation/up:catalyticActivity/up:catalyzedReaction ?rhea .\n   ?protein up:organism taxon:9606 .\n   # Find only those human entries that have an annotated related disease, and optionaly map these to OMIM\n   ?protein up:annotation/up:disease ?disease .\n   OPTIONAL {\n     ?disease rdfs:seeAlso ?omim .\n     ?omim up:database <http://purl.uniprot.org/database/MIM>\n   }\n}",
    "target_endpoint": "https://sparql.uniprot.org/sparql/",
    "federates_with": "https://idsm.elixir-czech.cz/sparql/endpoint/chebi",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/60_enzymes_working_on_substrate_with_Cholestene_backbone.ttl",
    "filename": "60_enzymes_working_on_substrate_with_Cholestene_backbone.ttl"
  },
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/70_enzymes_interacting_with_molecules_similar_to_dopamine",
    "natural_language_question": "Look for enzymes catalyzing reactions with molecules similar to dopamine.",
    "query": "PREFIX CHEBI: <http://purl.obolibrary.org/obo/CHEBI_>\nPREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX sachem: <http://bioinfo.uochb.cas.cz/rdf/v1.0/sachem#>\nPREFIX up: <http://purl.uniprot.org/core/>\n\nSELECT\n  ?protein\n  ?rhea\n  ?chebi\nWHERE {\n  # Use the smiles of dopamine CHEBI:18243\n  SERVICE <https://idsm.elixir-czech.cz/sparql/endpoint/chebi>{\n    ?chebi sachem:similarCompoundSearch [\n      sachem:query \"NCCc1ccc(O)c(O)c1\" ] .\n  }\n  GRAPH<https://sparql.rhea-db.org/rhea>{\n    ?rhea rh:side/rh:contains/rh:compound ?compound .\n    ?compound (rh:chebi|(rh:reactivePart/rh:chebi)|(rh:underlyingChebi/rh:chebi)) ?chebi .\n  }\n  ?protein up:reviewed true ;\n    up:annotation ?caa .\n  ?caa up:catalyticActivity/up:catalyzedReaction ?rhea .\n}",
    "target_endpoint": "https://sparql.uniprot.org/sparql/",
    "federates_with": "https://idsm.elixir-czech.cz/sparql/endpoint/chebi",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/70_enzymes_interacting_with_molecules_similar_to_dopamine.ttl",
    "filename": "70_enzymes_interacting_with_molecules_similar_to_dopamine.ttl"
  },
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/38",
    "natural_language_question": "For the human entry P05067 (Amyloid-beta precursor protein) find the gene start ends in WikiData",
    "query": "PREFIX p: <http://www.wikidata.org/prop/>\nPREFIX pq: <http://www.wikidata.org/prop/qualifier/>\nPREFIX ps: <http://www.wikidata.org/prop/statement/>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX uniprotkb: <http://purl.uniprot.org/uniprot/>\nPREFIX wdt: <http://www.wikidata.org/prop/direct/>\n\n\nSELECT \n\t?protein \n\t?begin\n\t?end\n\t?chromosome\n\t?assembly\nWHERE {\n    {\n        BIND(uniprotkb:P05067 AS ?proteinIRI)\n        BIND (SUBSTR(STR(?proteinIRI), STRLEN(STR(uniprotkb:))+1) AS ?protein)\n    }\n    SERVICE <https://query.wikidata.org/sparql> {\n        ?wp wdt:P352 ?protein ;\n            wdt:P702 ?wg . \n        ?wg p:P644   ?wgss .\n        ?wgss ps:P644        ?begin ;\n          pq:P1057/wdt:P1813 ?chromosome ;\n          pq:P659/rdfs:label ?assembly .\n        ?wg p:P645 ?wgse .\n        ?wgse ps:P645        ?end ;\n          pq:P1057/wdt:P1813 ?chromosome ;\n          pq:P659/rdfs:label ?assembly .\n        FILTER(lang(?assembly) = \"en\")\n  } \n}",
    "target_endpoint": "https://sparql.uniprot.org/sparql/",
    "federates_with": "https://query.wikidata.org/sparql",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/38_find_gene_region_in_wikidata_for_known_entry.ttl",
    "filename": "38_find_gene_region_in_wikidata_for_known_entry.ttl"
  },
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/117_biosodafrontend_glioblastoma_orthologs_rat",
    "natural_language_question": "Which are the proteins associated with glioblastoma and the orthologs expressed in the rat brain?",
    "query": "PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX obo: <http://purl.obolibrary.org/obo/>\nPREFIX orth: <http://purl.org/net/orth#>\nPREFIX sio: <http://semanticscience.org/resource/>\nPREFIX taxon: <http://purl.uniprot.org/taxonomy/>\nPREFIX up: <http://purl.uniprot.org/core/>\nPREFIX lscr: <http://purl.org/lscr#>\nPREFIX genex: <http://purl.org/genex#>\nSELECT DISTINCT ?protein ?orthologous_protein ?gene ?annotation_text WHERE {\n  {\n  \tSELECT ?protein ?annotation_text WHERE {\n      ?protein a up:Protein ;\n          up:organism taxon:9606 ;\n          up:annotation ?annotation .\n      ?annotation rdfs:comment ?annotation_text .\n      ?annotation a up:Disease_Annotation .\n      FILTER CONTAINS (?annotation_text, \"glioblastoma\")\n    }\n  }\n  SERVICE <https://sparql.omabrowser.org/sparql/> {\n    SELECT ?orthologous_protein ?protein ?gene WHERE {\n    ?protein_OMA a orth:Protein .\n    ?orthologous_protein a orth:Protein .\n    ?cluster a orth:OrthologsCluster .\n    ?cluster orth:hasHomologousMember ?node1 .\n    ?cluster\n    orth:hasHomologousMember ?node2 .\n    ?node2 orth:hasHomologousMember* ?protein_OMA .\n    ?node1 orth:hasHomologousMember* ?orthologous_protein .\n    ?orthologous_protein orth:organism/obo:RO_0002162 taxon:10116 . # rattus norvegicus\n    ?orthologous_protein sio:SIO_010079 ?gene .\n    ?protein_OMA lscr:xrefUniprot ?protein .\n    FILTER(?node1 != ?node2)\n\t\t}\n\t}\n  SERVICE <https://www.bgee.org/sparql/> {\n    ?gene genex:isExpressedIn ?a .\n    ?a rdfs:label \"brain\" .\n    ?gene orth:organism ?s . \n    ?s obo:RO_0002162 taxon:10116.\n\t}\n}",
    "target_endpoint": "https://sparql.uniprot.org/sparql",
    "federates_with": "https://sparql.omabrowser.org/sparql/",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/117_biosodafrontend_glioblastoma_orthologs_rat.ttl",
    "filename": "117_biosodafrontend_glioblastoma_orthologs_rat.ttl"
  },
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/116_biosodafrontend_rabit_mouse_orthologs",
    "natural_language_question": "Rabbit's proteins encoded by genes that are orthologous to Mouse's HBB-Y gene and their cross reference links to Uniprot",
    "query": "PREFIX lscr: <http://purl.org/lscr#>\nPREFIX orth: <http://purl.org/net/orth#>\nPREFIX up: <http://purl.uniprot.org/core/>\nPREFIX obo: <http://purl.obolibrary.org/obo/>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nSELECT DISTINCT ?PROTEIN_1 ?PROTEIN_2 ?UNIPROT_XREF_1 ?UNIPROT_XREF_2 WHERE {\n\t?taxon_1 up:commonName 'Mouse' .\n\t?taxon_2 up:commonName 'Rabbit' .\n\tSERVICE <https://sparql.omabrowser.org/sparql/> {\n\t\t?cluster a orth:OrthologsCluster .\n\t\t?cluster orth:hasHomologousMember ?node1 .\n\t\t?cluster orth:hasHomologousMember ?node2 .\n\t\t?node2 orth:hasHomologousMember* ?PROTEIN_2 .\n\t\t?node1 orth:hasHomologousMember* ?PROTEIN_1 .\n\t\t?PROTEIN_1 a orth:Protein .\n\t\t?PROTEIN_1 orth:organism/obo:RO_0002162 ?taxon_1 ;\n\t\t\trdfs:label 'HBB-Y' ;\n\t\t\tlscr:xrefUniprot ?UNIPROT_XREF_1 .\n\t\t?PROTEIN_2 a orth:Protein .\n\t\t?PROTEIN_2 orth:organism/obo:RO_0002162 ?taxon_2 .\n\t\t?PROTEIN_2 lscr:xrefUniprot ?UNIPROT_XREF_2 .\n\t\tFILTER ( ?node1 != ?node2 )\n\t}\n}",
    "target_endpoint": "https://sparql.uniprot.org/sparql",
    "federates_with": "https://sparql.omabrowser.org/sparql/",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/116_biosodafrontend_rabit_mouse_orthologs.ttl",
    "filename": "116_biosodafrontend_rabit_mouse_orthologs.ttl"
  },
  {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/29",
    "natural_language_question": "Use <a href=\"http://allie.dbcls.jp/\">ALLIE</a> a service for Abbreviation / Long Form in Japanese and English to search in UniProt using Japanese.",
    "query": "PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX up: <http://purl.uniprot.org/core/>\n\nSELECT ?protein ?englishLabelStr\nWHERE {\n    SERVICE <https://data.allie.dbcls.jp/sparql>{\n        ?x rdfs:label \"アミロイド前駆体タンパク質\"@ja ;\n            rdfs:label ?englishLabel .\n        FILTER(lang(?englishLabel) = \"en\")\n    }\n    BIND (STR(?englishLabel) AS ?englishLabelStr)\n    ?protein a up:Protein .\n    {\n        ?protein (up:recommendedName|up:alternativeName) ?structuredName .\n    }\n    UNION\n    {\n        VALUES(?partType){(up:domain) (up:component)}\n            ?protein ?partType ?part .\n        ?part (up:recommendedName|up:alternativeName) ?structuredName .\n    }\n    ?structuredName ?anyKindOfName  ?englishLabelStr .\n    ?anyKindOfName rdfs:subPropertyOf up:structuredNameType .\n}",
    "target_endpoint": "https://sparql.uniprot.org/sparql/",
    "federates_with": "https://data.allie.dbcls.jp/sparql",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/29_search_uniprot_in_japanese_with_allie.ttl",
    "filename": "29_search_uniprot_in_japanese_with_allie.ttl"
  },
  {
    "resource": "https://sparql.rhea-db.org/.well-known/sparql-examples/94_Select_all_approved_reactions_with_CHEBI_or_one_of_its_descendant_optional_UniProtKB_reviewed_EC",
    "natural_language_question": "Select all approved reactions with CHEBI:17815 (a 1,2-diacyl-sn-glycerol) or one of its descendant. Display links to UniProtKB/Swiss-Prot entries via EC numbers if the link rhea-ec exists.",
    "query": "PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX rh:<http://rdf.rhea-db.org/>\nPREFIX CHEBI:<http://purl.obolibrary.org/obo/CHEBI_>\nPREFIX up:<http://purl.uniprot.org/core/>\n\nSELECT\n  distinct\n    ?reaction\n    ?enzyme\n    (count(distinct ?protein) as ?proteinCount)\nWHERE {\n  ?reaction rdfs:subClassOf rh:Reaction .\n  ?reaction rh:status rh:Approved .\n  ?reaction rh:side ?reactionSide .\n\n  OPTIONAL {?reaction rh:ec ?enzyme .\n    SERVICE <https://sparql.uniprot.org/sparql> {\n      ?protein up:reviewed true .\n      ?protein up:enzyme ?enzyme.\n    }\n  } .\n  ?reactionSide rh:contains ?participant .\n  ?participant rh:compound ?compound .\n  ?compound rh:chebi ?chebi .\n  ?chebi rdfs:subClassOf* CHEBI:17815 .\n}\nGROUP BY ?reaction ?enzyme\nORDER BY ?reaction",
    "target_endpoint": "https://sparql.rhea-db.org/sparql",
    "federates_with": "https://sparql.uniprot.org/sparql",
    "endpoint_set": "Rhea",
    "file_path": "Rhea/94_Select_all_approved_reactions_with_CHEBI_or_one_of_its_descendant_optional_UniProtKB_reviewed_EC.ttl",
    "filename": "94_Select_all_approved_reactions_with_CHEBI_or_one_of_its_descendant_optional_UniProtKB_reviewed_EC.ttl"
  },
  {
    "resource": "https://sparql.rhea-db.org/.well-known/sparql-examples/11",
    "natural_language_question": "Distribution of reactions according to the first class of the enzyme classification (federated query)",
    "query": "PREFIX ec: <http://purl.uniprot.org/enzyme/>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX skos: <http://www.w3.org/2004/02/skos/core#>\n\n# Query 11\n# Retrieve the count of reactions mapped to each level (main class) of the enzyme classification\n#\n# This query mimics the Filter section of the Rhea website (Browse all reactions)\n# https://www.rhea-db.org/rhea?query=\nSELECT ?ecClass (STR(?ecName) AS ?ecClassName) (COUNT(?rhea) AS ?rheaCount)\nWHERE {\n  SERVICE <https://sparql.uniprot.org/sparql> {\n    VALUES (?ecClass) { (ec:1.-.-.-)(ec:2.-.-.-)(ec:3.-.-.-)(ec:4.-.-.-)(ec:5.-.-.-) (ec:6.-.-.-) (ec:7.-.-.-)}\n    ?ecNumber rdfs:subClassOf ?ecClass .\n    ?ecClass skos:prefLabel ?ecName .\n  }\n  ?rhea rdfs:subClassOf rh:Reaction .\n  ?rhea rh:ec ?ecNumber .\n} GROUP BY ?ecClass ?ecName",
    "target_endpoint": "https://sparql.rhea-db.org/sparql",
    "federates_with": "https://sparql.uniprot.org/sparql",
    "endpoint_set": "Rhea",
    "file_path": "Rhea/11.ttl",
    "filename": "11.ttl"
  },
  {
    "resource": "https://sparql.rhea-db.org/.well-known/sparql-examples/109_Distribution_of_reactions_according_to_the_first_level_of_enzyme_classification",
    "natural_language_question": "Distribution of reactions according to the first level of enzyme classification",
    "query": "\nPREFIX rh:<http://rdf.rhea-db.org/>\nPREFIX ec:<http://purl.uniprot.org/enzyme/>\nPREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>\nPREFIX skos:<http://www.w3.org/2004/02/skos/core#>\n\nSELECT\n  ?ecClass\n  (str(?ecName) as ?ecClassName)\n  (count(?reaction) as ?reactionCount)\nWHERE\n{\n  SERVICE <http://sparql.uniprot.org/sparql> {\n    VALUES (?ecClass) { (ec:1.-.-.-)(ec:2.-.-.-)(ec:3.-.-.-)(ec:4.-.-.-)(ec:5.-.-.-) (ec:6.-.-.-) (ec:7.-.-.-) }\n    ?ecNumber rdfs:subClassOf ?ecClass .\n    ?ecClass skos:prefLabel ?ecName .\n  }\n  ?reaction rdfs:subClassOf rh:Reaction .\n  ?reaction rh:ec ?ecNumber .\n}\nGROUP BY ?ecClass ?ecName\n",
    "target_endpoint": "https://sparql.rhea-db.org/sparql",
    "federates_with": "http://sparql.uniprot.org/sparql",
    "endpoint_set": "Rhea",
    "file_path": "Rhea/109_Distribution_of_reactions_according_to_the_first_level_of_enzyme_classification.ttl",
    "filename": "109_Distribution_of_reactions_according_to_the_first_level_of_enzyme_classification.ttl"
  },
  {
    "resource": "https://sparql.rhea-db.org/.well-known/sparql-examples/12",
    "natural_language_question": "Select all Rhea reactions linked to protein sequences (enzymes and transporters) in UniProtKB",
    "query": "PREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX up: <http://purl.uniprot.org/core/>\n\n# Query 12\n# Select all Rhea reactions used to annotate enzyme sequences in UniProtKB\n# return the number of UniProtKB entries\n# Federated query using a service to UniProt SPARQL endpoint\n#\n# This query corresponds to the Rhea website query:\n# https://www.rhea-db.org/rhea?query=uniprot:*\n#\nSELECT ?uniprotCount ?rhea ?accession ?equation \nWHERE {\n  SERVICE <https://sparql.uniprot.org/sparql> { \n  \tSELECT ?rhea (count(?uniprot) as ?uniprotCount) {\n      ?uniprot up:annotation/up:catalyticActivity/up:catalyzedReaction ?rhea . \n  \t}\n  \tGROUP BY ?rhea\n  }\n  ?rhea rh:accession ?accession .\n  ?rhea rh:equation ?equation .\n}",
    "target_endpoint": "https://sparql.rhea-db.org/sparql",
    "federates_with": "https://sparql.uniprot.org/sparql",
    "endpoint_set": "Rhea",
    "file_path": "Rhea/12.ttl",
    "filename": "12.ttl"
  },
  {
    "resource": "https://sparql.rhea-db.org/.well-known/sparql-examples/112_Total_number_of_reaction_ec_protein_links",
    "natural_language_question": "Total number of <reaction> <ec> <protein> links",
    "query": "PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX rh:<http://rdf.rhea-db.org/>\nPREFIX ec:<http://purl.uniprot.org/enzyme/>\nPREFIX up:<http://purl.uniprot.org/core/>\n\nSELECT\n  (count(?reaction) as ?reactionEcProteinLinkCount)\nWHERE {\n  ?reaction rdfs:subClassOf rh:Reaction .\n  ?reaction rh:ec ?enzyme\n  SERVICE <http://sparql.uniprot.org/sparql> {\n    ?protein up:reviewed true .\n    ?protein up:enzyme ?enzyme.\n  }\n}",
    "target_endpoint": "https://sparql.rhea-db.org/sparql",
    "federates_with": "http://sparql.uniprot.org/sparql",
    "endpoint_set": "Rhea",
    "file_path": "Rhea/112_Total_number_of_reaction_ec_protein_links.ttl",
    "filename": "112_Total_number_of_reaction_ec_protein_links.ttl"
  },
  {
    "resource": "https://sparql.rhea-db.org/.well-known/sparql-examples/19_draft_human_metabolome",
    "natural_language_question": "Generate a draft human metabolome",
    "query": "PREFIX chebislash: <http://purl.obolibrary.org/obo/chebi/>\nPREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX taxon: <http://purl.uniprot.org/taxonomy/>\nPREFIX up: <http://purl.uniprot.org/core/>\n\nSELECT\n  ?uniprot ?mnemonic ?rhea ?chebi ?smiles ?inchiKey\nWHERE\n{\n  ?rhea rh:side/rh:contains/rh:compound ?compound .\n  ?compound (rh:chebi|(rh:reactivePart/rh:chebi)|(rh:underlyingChebi/rh:chebi)) ?chebi .\n  ?chebi chebislash:smiles ?smiles ;\n          chebislash:inchikey ?inchiKey .\n  SERVICE <https://sparql.uniprot.org/sparql> {\n     ?uniprot up:annotation/up:catalyticActivity/up:catalyzedReaction ?rhea ;\n                                             up:organism taxon:9606 ;\n                                             up:mnemonic ?mnemonic .\n\n   }\n}",
    "target_endpoint": "https://sparql.rhea-db.org/sparql/",
    "federates_with": "https://sparql.uniprot.org/sparql",
    "endpoint_set": "Rhea",
    "file_path": "Rhea/19_draft_human_metabolome.ttl",
    "filename": "19_draft_human_metabolome.ttl"
  },
  {
    "resource": "https://sparql.rhea-db.org/.well-known/sparql-examples/20_search_chemical_names_in_japanese",
    "natural_language_question": "Search for Chemical names starting in Japanese, translated to english using allie",
    "query": "PREFIX oboInOwl: <http://www.geneontology.org/formats/oboInOwl#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\n\nSELECT\n*\nWHERE {\nSERVICE <https://data.allie.dbcls.jp/sparql>{\n        ?x rdfs:label \"1,2,4-トリクロロベンゼン\"@ja ;\n            rdfs:label ?englishLabel .\n        FILTER(lang(?englishLabel) = \"en\")\n    }\n  BIND(STR(?englishLabel) AS ?englishLabelStr)\n  ?chebi rdfs:label|oboInOwl:hasSynonym ?englishLabelStr .\n}",
    "target_endpoint": "https://sparql.rhea-db.org/sparql/",
    "federates_with": "https://data.allie.dbcls.jp/sparql",
    "endpoint_set": "Rhea",
    "file_path": "Rhea/20_search_chemical_names_in_japanese.ttl",
    "filename": "20_search_chemical_names_in_japanese.ttl"
  },
  {
    "resource": "https://sparql.rhea-db.org/.well-known/sparql-examples/117_Retrieve_parent_reactions_and_their_catalyzing_enzymes_for_a_metabolite_that_is_not_directly_involved_in_a_reaction",
    "natural_language_question": "Retrieve parent reactions (and their catalyzing enzymes) for a metabolite that is not directly involved in a reaction (ex: CHEBI:83137).",
    "query": "PREFIX CHEBI: <http://purl.obolibrary.org/obo/CHEBI_>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX rh:<http://rdf.rhea-db.org/>\nPREFIX ec:<http://purl.uniprot.org/enzyme/>\nPREFIX up:<http://purl.uniprot.org/core/>\nPREFIX taxon:<http://purl.uniprot.org/taxonomy/>\nPREFIX keywords:<http://purl.uniprot.org/keywords/>\n\nSELECT\n  ?chebi\n  ?ecNumber\n  (count(?protein) as ?proteinCount)\n  ?reaction\n  ?equation\nWHERE {\n  ?reaction rdfs:subClassOf rh:Reaction .\n  ?reaction rh:status rh:Approved .\n  ?reaction rh:equation ?equation .\n  ?reaction rh:ec ?ecNumber .\n  ?reaction rh:side ?reactionSide .\n  ?reactionSide rh:contains ?participant .\n  ?participant rh:compound ?compound .\n  ?compound rh:chebi ?chebi .\n  CHEBI:83137 rdfs:subClassOf+ ?chebi.\n\n  SERVICE <https://sparql.uniprot.org/sparql> {\n  ?protein a up:Protein .\n  ?protein up:reviewed true .\n    {?protein up:enzyme ?ecNumber}\n  UNION\n    {?protein up:domain/up:enzyme ?ecNumber}\n  UNION\n    {?protein up:component/up:enzyme ?ecNumber} .\n  }\n} \nGROUP BY ?chebi ?ecNumber ?reaction ?equation",
    "target_endpoint": "https://sparql.rhea-db.org/sparql",
    "federates_with": "https://sparql.uniprot.org/sparql",
    "endpoint_set": "Rhea",
    "file_path": "Rhea/117_Retrieve_parent_reactions_and_their_catalyzing_enzymes_for_a_metabolite_that_is_not_directly_involved_in_a_reaction.ttl",
    "filename": "117_Retrieve_parent_reactions_and_their_catalyzing_enzymes_for_a_metabolite_that_is_not_directly_involved_in_a_reaction.ttl"
  },
  {
    "resource": "https://sparql.rhea-db.org/.well-known/sparql-examples/18",
    "natural_language_question": "Use <a href=\"https://idsm.elixir-czech.cz/sparql\">IDSM Sachem</a> to find ChEBIs with a a <a href=\"https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI:35516\">Cholestane</a> skeleton (in SMILES). Then match returned ChEBIs to Rhea undirected reactions.",
    "query": "PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX sachem: <http://bioinfo.uochb.cas.cz/rdf/v1.0/sachem#>\n\n\nSELECT \n  ?rhea \n  ?chebi\nWHERE {\n  SERVICE <https://idsm.elixir-czech.cz/sparql/endpoint/chebi> {\n    ?chebi sachem:substructureSearch [\n        sachem:query \"[C@]12(CCC3CCCC[C@]3(C)[C@@]1([H])CC[C@]1(C)[C@@]([H])([C@@](C)([H])CCCC(C)C)CC[C@@]21[H])[H]\" ].\n  }\n  ?rhea rh:side/rh:contains/rh:compound/rdfs:subClassOf ?chebi .\n}",
    "target_endpoint": "https://sparql.rhea-db.org/sparql",
    "federates_with": "https://idsm.elixir-czech.cz/sparql/endpoint/chebi",
    "endpoint_set": "Rhea",
    "file_path": "Rhea/18.ttl",
    "filename": "18.ttl"
  },
  {
    "resource": "https://sparql.rhea-db.org/.well-known/sparql-examples/115_Retrieve_the_set_of_metabolites_involved_in_reactions_catalyzed_by_human_genes_HUMAN_reference_proteome",
    "natural_language_question": "Retrieve the set of metabolites involved in reactions catalyzed by human genes (HUMAN reference proteome)",
    "query": "\nPREFIX rh:<http://rdf.rhea-db.org/>\nPREFIX taxon:<http://purl.uniprot.org/taxonomy/>\nPREFIX ec:<http://purl.uniprot.org/enzyme/>\nPREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>\nPREFIX up:<http://purl.uniprot.org/core/>\nPREFIX keywords:<http://purl.uniprot.org/keywords/>\n\nSELECT\n  ?chebi\n  ?reaction\n  ?ecNumber\n  ?protein\n  ?ensemblTranscript\n  ?ensemblGene\nWHERE {\n  SERVICE <https://sparql.uniprot.org/sparql> {\n    ?protein a up:Protein .\n    ?protein up:reviewed true .\n    ?protein up:organism taxon:9606 .\n    ?protein up:classifiedWith keywords:1185 .\n\n    ?protein rdfs:seeAlso ?ensemblTranscript .\n    ?ensemblTranscript a up:Transcript_Resource .\n    ?ensemblTranscript up:database <http://purl.uniprot.org/database/Ensembl> .\n    ?ensemblTranscript up:transcribedFrom ?ensemblGene .\n\n      {?protein up:enzyme ?ecNumber .}\n    UNION\n      {?protein up:domain/up:enzyme ?ecNumber .}\n    UNION\n      {?protein up:component/up:enzyme ?ecNumber .}\n  }\n\n  ?reaction rdfs:subClassOf rh:Reaction .\n  ?reaction rh:status rh:Approved .\n  ?reaction rh:ec ?ecNumber .\n  ?reaction rh:side ?reactionSide .\n  ?reactionSide rh:contains ?participant .\n  ?participant rh:compound ?compound .\n  ?compound rh:chebi ?chebi . # only considering small molecules participants\n}",
    "target_endpoint": "https://sparql.rhea-db.org/sparql",
    "federates_with": "https://sparql.uniprot.org/sparql",
    "endpoint_set": "Rhea",
    "file_path": "Rhea/115_Retrieve_the_set_of_metabolites_involved_in_reactions_catalyzed_by_human_genes_HUMAN_reference_proteome.ttl",
    "filename": "115_Retrieve_the_set_of_metabolites_involved_in_reactions_catalyzed_by_human_genes_HUMAN_reference_proteome.ttl"
  },
  {
    "resource": "https://sparql.swisslipids.org/.well-known/sparql-examples/16",
    "natural_language_question": "For a given lipid (SWISSLIPID:000399814, Ceramide), return a list of all enzymes in UniProtKB/Swiss-Prot annotated to catalyze reactions involving that lipid.\nThis query does not consider the hierarchy: it looks for reactions and enzymes only for the specified lipid.",
    "query": "PREFIX SWISSLIPID: <https://swisslipids.org/rdf/SLM_>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX up: <http://purl.uniprot.org/core/>\n\n# Example 16\nSELECT ?startId ?startName ?uniprot\nWHERE\n{\n  # SwissLipids query (list of) identifier(s)\n  VALUES ?startId { SWISSLIPID:000399814 }\n  # name\n  ?startId rdfs:label ?startName .\n  # ChEBI x-ref\n  ?startId owl:equivalentClass ?chebi .\n  # federated query to Rhea\n  SERVICE <https://sparql.rhea-db.org/sparql> {\n  \t?rhea rh:side/rh:contains/rh:compound/rh:chebi ?chebi .\n  }    \n  # federated query to UniProt\n  SERVICE <https://sparql.uniprot.org/sparql> {\n    ?uniprot up:reviewed true .\n    ?uniprot up:annotation/up:catalyticActivity/up:catalyzedReaction ?rhea .\n  }\n} \nORDER BY ?startId",
    "target_endpoint": "https://sparql.swisslipids.org/sparql/",
    "federates_with": "https://sparql.uniprot.org/sparql",
    "endpoint_set": "SwissLipids",
    "file_path": "SwissLipids/16.ttl",
    "filename": "16.ttl"
  },
  {
    "resource": "https://sparql.swisslipids.org/.well-known/sparql-examples/6",
    "natural_language_question": "For a given lipid (SWISSLIPID:000399814, Ceramide), return a list of all corresponding reactions (Rhea IDs and equation), as well as their enzymes as annotated in UniProtKB/Swiss-Prot) enzymes themselves (ID and protein name).\nThis query does not consider the hierarchy: it looks for reactions and enzymes only for the specified lipid.",
    "query": "PREFIX SWISSLIPID: <https://swisslipids.org/rdf/SLM_>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX up: <http://purl.uniprot.org/core/>\n\n# Example 6\nSELECT ?startId ?startName ?chebi ?rhea ?rheaEquation ?uniprot ?uniprotName\nWHERE\n{\n  # SwissLipids query (list of) identifier(s)\n  VALUES ?startId { SWISSLIPID:000399814 }\n  # name\n  ?startId rdfs:label ?startName .\n  # ChEBI x-ref\n  ?startId owl:equivalentClass ?chebi .\n  # federated query to Rhea\n  SERVICE <https://sparql.rhea-db.org/sparql> {\n  \t?rhea rh:equation ?rheaEquation .\n  \t?rhea rh:side/rh:contains/rh:compound/rh:chebi ?chebi .\n  }    \n  # federated query to UniProt\n  SERVICE <https://sparql.uniprot.org/sparql> {\n    ?uniprot up:reviewed true .\n    ?uniprot up:recommendedName/up:fullName ?uniprotName .\n    ?uniprot up:annotation/up:catalyticActivity/up:catalyzedReaction ?rhea .\n  }\n} \nORDER BY ?startId",
    "target_endpoint": "https://sparql.swisslipids.org/sparql/",
    "federates_with": "https://sparql.uniprot.org/sparql",
    "endpoint_set": "SwissLipids",
    "file_path": "SwissLipids/6.ttl",
    "filename": "6.ttl"
  },
  {
    "resource": "https://sparql.swisslipids.org/.well-known/sparql-examples/5",
    "natural_language_question": "For a given lipid (SWISSLIPID:000399814, Ceramide), return a list of all corresponding reactions (Rhea IDs and equation).\nThis query does consider the hierarchy: it looks for reactions involving the specified lipid and/or its more specific forms.",
    "query": "PREFIX SWISSLIPID: <https://swisslipids.org/rdf/SLM_>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX rh: <http://rdf.rhea-db.org/>\n\n# Example 5\nSELECT ?startId ?startName ?id ?name ?chebi ?rhea ?rheaEquation \nWHERE\n{\n  # SwissLipids query (list of) identifier(s)\n  VALUES ?startId { SWISSLIPID:000399814 }\n  # name\n  ?startId rdfs:label ?startName .\n  # Retrieve children lipids \n  ?id rdfs:subClassOf* ?startId .\n  ?id rdfs:label ?name .\n  # ChEBI x-ref\n  ?id owl:equivalentClass ?chebi .\n  # federated query to Rhea\n  SERVICE <https://sparql.rhea-db.org/sparql> {\n  \t?rhea rh:equation ?rheaEquation .\n  \t?rhea rh:side/rh:contains/rh:compound/rh:chebi ?chebi .\n  }    \n} \nORDER BY  ?startId ?id",
    "target_endpoint": "https://sparql.swisslipids.org/sparql/",
    "federates_with": "https://sparql.rhea-db.org/sparql",
    "endpoint_set": "SwissLipids",
    "file_path": "SwissLipids/5.ttl",
    "filename": "5.ttl"
  },
  {
    "resource": "https://sparql.swisslipids.org/.well-known/sparql-examples/34",
    "natural_language_question": "Lipids affected by human enzymes",
    "query": "PREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX taxon: <http://purl.uniprot.org/taxonomy/>\nPREFIX up: <http://purl.uniprot.org/core/>\n\nSELECT ?swisslipid  {\n  ?swisslipid owl:equivalentClass ?chebi .\n  SERVICE <https://sparql.rhea-db.org/sparql> {\n    ?rhea rh:side/rh:contains/rh:compound ?compound .\n\t?compound (rh:chebi|(rh:reactivePart/rh:chebi)|(rh:underlyingChebi/rh:chebi)) ?metabolite . \n  }\n  SERVICE <https://sparql.uniprot.org/sparql> {\n    ?catalyticActivityAnnotation up:catalyticActivity/up:catalyzedReaction ?rhea .\n    ?protein up:annotation ?catalyticActivityAnnotation ;\n             up:organism taxon:9606 .\n  }\n}",
    "target_endpoint": "https://sparql.swisslipids.org/sparql/",
    "federates_with": "https://sparql.uniprot.org/sparql",
    "endpoint_set": "SwissLipids",
    "file_path": "SwissLipids/34.ttl",
    "filename": "34.ttl"
  },
]

In [ ]:

error_list = []
no_result_list = []
count = 0

for object in test_set:
    print(f"Querying endpoint {object['filename']}...")
    print(f"arrow {count}/{len(test_set)}")
    result = query_sparql(object['query'] + " LIMIT 1", object['target_endpoint'])
    count += 1
    if result == 'error':
        print(f"Error querying endpoint {object['filename']}")
        error_list.append({"filename": object['filename'],"query": object['query'],"endpoint": object['target_endpoint'], "result": "error"})
    else:
        if not result.get("results") or not result["results"].get("bindings") or len(result["results"]["bindings"]) == 0:
            print(f"No results for {object['filename']}")
            no_result_list.append({
                "filename": object['filename'],
                "query": object['query'],
                "endpoint": object['target_endpoint'],
                "result": "no_results"
            })


In [ ]:
print(len(error_list))

In [ ]:
df_ground_truth_errors= pd.DataFrame(error_list)

In [ ]:
df_ground_truth_errors

In [2]:
import sys
import os


sys.path.append('/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent')

from experiments.utilities.query_cache import cache_dataset_queries


result = cache_dataset_queries(
    endpoint_files_map={
        "UniProt": [], 
        "Rhea": [],
        "SwissLipids": []
    }
)

Processing endpoint set: UniProt
  Processing all files in UniProt
Processing TTL file: https://sparql.uniprot.org/.well-known/sparql-examples/118_biosodafrontend_rat_brain_human_cancer
  Using cached result for query
Processing TTL file: https://sparql.uniprot.org/.well-known/sparql-examples/51
  Using cached result for query
Processing TTL file: https://sparql.uniprot.org/.well-known/sparql-examples/40
  Using cached result for query
Processing TTL file: https://sparql.uniprot.org/.well-known/sparql-examples/49
  Using cached result for query
Processing TTL file: https://sparql.uniprot.org/.well-known/sparql-examples/53
  Using cached result for query
Processing TTL file: https://sparql.uniprot.org/.well-known/sparql-examples/52
  Using cached result for query
Processing TTL file: https://sparql.uniprot.org/.well-known/sparql-examples/54
  Using cached result for query
Processing TTL file: https://sparql.uniprot.org/.well-known/sparql-examples/43
  Using cached result for query
Proce

In [ ]:
import sys
import os
sys.path.append('/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent')

from experiments.utilities.query_cache import query_sparql

test = {
    "resource": "https://sparql.uniprot.org/.well-known/sparql-examples/67_draft_human_metabolome",
    "natural_language_question": "Generate a draft human metabolome",
    "query": "PREFIX chebislash: <http://purl.obolibrary.org/obo/chebi/>\nPREFIX rh: <http://rdf.rhea-db.org/>\nPREFIX taxon: <http://purl.uniprot.org/taxonomy/>\nPREFIX up: <http://purl.uniprot.org/core/>\n\nSELECT\n?uniprot ?mnemonic ?rhea ?chebi ?smiles ?inchiKey\nWHERE\n{\n  ?uniprot up:annotation/up:catalyticActivity/up:catalyzedReaction ?rhea ;\n        up:organism taxon:9606 ;\n        up:mnemonic ?mnemonic .\n  SERVICE <https://sparql.rhea-db.org/sparql> {\n\t?rhea rh:side/rh:contains/rh:compound ?compound .\n    ?compound (rh:chebi|(rh:reactivePart/rh:chebi)|(rh:underlyingChebi/rh:chebi)) ?chebi .\n   }\n   ?chebi chebislash:smiles ?smiles ;\n          chebislash:inchikey ?inchiKey .\n}\nLIMIT 10",
    "target_endpoint": "https://sparql.uniprot.org/sparql/",
    "federates_with": "https://sparql.rhea-db.org/sparql",
    "endpoint_set": "Uniprot",
    "file_path": "UniProt/67_draft_human_metabolome.ttl",
    "filename": "67_draft_human_metabolome.ttl"
  }

result = query_sparql(test['query'], test['target_endpoint'])

print(result)
